# urls
https://raw.githubusercontent.com/CriMenghini/ADM/master/2019/Homework_3/data/movies3.html


https://raw.githubusercontent.com/CriMenghini/ADM/master/2019/Homework_3/data/movies2.html
    
https://raw.githubusercontent.com/CriMenghini/ADM/master/2019/Homework_3/data/movies1.html

In [1]:
urls = [' https://raw.githubusercontent.com/CriMenghini/ADM/master/2019/Homework_3/data/movies3.html',
        'https://raw.githubusercontent.com/CriMenghini/ADM/master/2019/Homework_3/data/movies2.html',
        'https://raw.githubusercontent.com/CriMenghini/ADM/master/2019/Homework_3/data/movies1.html']

# Import

In [2]:
import requests
from bs4 import BeautifulSoup
import random
import time
import re
from lxml import etree
import lxml.html
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize

# Extract movie links

In [3]:
movie_links = []
for url in urls:
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        for link in soup.find_all('a',href=True):
            movie_links.append(link['href'])
    except Exception as ex:
        print('error is ', ex)

In [4]:
len(movie_links)

30000

In [5]:
movie_links[:5]

['https://en.wikipedia.org/wiki/Z.P.G.',
 'https://en.wikipedia.org/wiki/Zee_and_Co.',
 'https://en.wikipedia.org/wiki/And_Now_the_Screaming_Starts!',
 'https://en.wikipedia.org/wiki/The_Asphyx',
 'https://en.wikipedia.org/wiki/Assassin_(1973_film)']

# Stopwords

In [30]:
tokens_without_sw_intro = []
def rm_stopwords(text):
    new_text = text.lower()
    global tokens_without_sw_intro
    stop_words = set(stopwords.words('english')) 
    text_tokens_intro = word_tokenize(new_text)
    tokens_without_sw_intro = [word for word in text_tokens_intro if not word in stopwords.words()]
   
    return tokens_without_sw_intro
    

# Punctuation

In [31]:
new_words_intro =[]
def rm_punctuation ():
    global new_words_intro
    tokenizer = nltk.RegexpTokenizer(r"\w+")
    punc_intro = ",".join(tokens_without_sw_intro)
    new_words_intro = tokenizer.tokenize(punc_intro)
    return new_words_intro

# Stemming

In [49]:
# streaming_intro = []
def stemming():
    global streaming_intro
    streaming_intro = []
    ps = PorterStemmer()
    for x in new_words_intro:
        
        streaming_intro.append(ps.stem(x))
    return streaming_intro

# Final normalize

In [53]:
# final_intro = []
def Final_normalize():
    global streaming_intro
    final_intro = []
    for i in streaming_intro:
        if len(i) > 3:
            final_intro.append(i)
    return " ".join(final_intro)

# Title

In [70]:
for t in movie_links[:3]:
    web_response = requests.get(t)
    soup = BeautifulSoup(web_response.text, 'html.parser')
    title = soup.title.text.split('-')
    title = title[0]
    print(title)

Z.P.G. 
Zee and Co. 
And Now the Screaming Starts! 


# Intro

In [58]:
for i in movie_links[:3]:
#     print(i)
    response = requests.get(i) 
    soup = BeautifulSoup(response.text, 'html.parser')
    intro =  soup.find('div' , { "id" : "toc" }).find_all_previous('p')[::-1]
    l=[]
    for i in intro:
        x = i.text.strip()
        if x != '':
            l.append(x)
    y= " ".join(l)
    # remove stopwords
    rm_stopword_intro = rm_stopwords(y)
    #remove punchtuation
    rm_punc_intro=rm_punctuation()

   # stemmer
    steming_intro = stemming()
   
    # final_normalize
    final_norm_intro = Final_normalize()


# plot

In [65]:
for p in movie_links[:3]: 
    response = requests.get(p) 
    soup = BeautifulSoup(response.text, 'html.parser')
    plot =  soup.find('span' , { "class" : "mw-editsection-bracket" }).find_next('p')
    final_plot = plot.text.strip()
    # remove stopwords
    rm_stopword = rm_stopwords(final_plot)
    
    # remove punch
    rm_punc=rm_punctuation()

    # stemmer
    steming = stemming()
    
    #final_normalize
    final_norm = Final_normalize()
    

# Table

In [74]:
for a in movie_links[:3]:
    response = requests.get(a) 
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find('table', class_ = 'infobox vevent')
    element_tree = lxml.html.fromstring(response.text)
    check_list = ['movie name','Directed by','Produced by',
                  'Written by','Starring','Music by',
                  'Release date','Running time','Country',
                  'Language','Budget']
    info = []

    vocabulary=[]

    stars ='//*[@id="mw-content-text"]/div[1]/table/tbody/tr[6]/td/div/ul/li/*/text()'
    writen ='//*[@id="mw-content-text"]/div[1]/table/tbody/tr[5]/td//a/text()'
    Path =[
        '//*[@id="mw-content-text"]/div[1]/table/tbody/tr[1]/th/text()',
        '//*[@id="mw-content-text"]/div[1]/table/tbody/tr[3]/td/a/text()',
        '//*[@id="mw-content-text"]/div[1]/table/tbody/tr[4]/td/text()',
        writen,
        stars,
        '//*[@id="mw-content-text"]/div[1]/table/tbody/tr[7]/td/a/text()',
        '//*[@id="mw-content-text"]/div[1]/table/tbody/tr[12]/td/div/ul/li/text()',
        '//*[@id="mw-content-text"]/div[1]/table/tbody/tr[13]/td/text()',
        '//*[@id="mw-content-text"]/div[1]/table/tbody/tr[14]/td/text()',
        '//*[@id="mw-content-text"]/div[1]/table/tbody/tr[15]/td/text()',
        '//*[@id="mw-content-text"]/div[1]/table/tbody/tr[16]/td/text()'

    ]
    for i in Path:
        if element_tree.xpath(i):
            info.append(element_tree.xpath(i))
            vocabulary.append(element_tree.xpath(i))
        else:
            info.append("NA")

    dictionaty= dict(zip([x for x in range(len(vocabulary))],vocabulary))
    print(dictionaty)

In [75]:
print(dictionaty)

{0: ['10 to Midnight'], 1: ['J. Lee Thompson'], 2: ['Pancho Kohner', 'Lance Hool'], 3: ['William Roberts', 'J. Lee Thompson'], 4: ['Charles Bronson', 'Lisa Eilbacher', 'Andrew Stevens', 'Gene Davis', 'Geoffrey Lewis', 'Wilford Brimley'], 5: ['Robert O. Ragland'], 6: ['March\xa011,\xa01983', ' (United States)'], 7: ['101 minutes'], 8: ['United States'], 9: ['English'], 10: ['$4,520,000 (US)']}


In [76]:
print(vocabulary)

[['10 to Midnight'], ['J. Lee Thompson'], ['Pancho Kohner', 'Lance Hool'], ['William Roberts', 'J. Lee Thompson'], ['Charles Bronson', 'Lisa Eilbacher', 'Andrew Stevens', 'Gene Davis', 'Geoffrey Lewis', 'Wilford Brimley'], ['Robert O. Ragland'], ['March\xa011,\xa01983', ' (United States)'], ['101 minutes'], ['United States'], ['English'], ['$4,520,000 (US)']]


In [77]:
print(info)

[['10 to Midnight'], ['J. Lee Thompson'], ['Pancho Kohner', 'Lance Hool'], ['William Roberts', 'J. Lee Thompson'], ['Charles Bronson', 'Lisa Eilbacher', 'Andrew Stevens', 'Gene Davis', 'Geoffrey Lewis', 'Wilford Brimley'], ['Robert O. Ragland'], ['March\xa011,\xa01983', ' (United States)'], ['101 minutes'], ['United States'], ['English'], ['$4,520,000 (US)']]


# Table information with lxml.xpath

In [128]:
for p in movie_links[:3]:
    try:
        response = requests.get(p) 
        element_tree = lxml.html.fromstring(response.text)
        Directed = element_tree.xpath('//*[@id="mw-content-text"]/div[1]/table/tbody/tr[3]/td//text()')
        Produced  = element_tree.xpath('//*[@id="mw-content-text"]/div[1]/table/tbody/tr[4]/td//text()')
        Written = element_tree.xpath('//*[@id="mw-content-text"]/div[1]/table/tbody/tr[5]/td//text()')
        Starring = element_tree.xpath('//*[@id="mw-content-text"]/div[1]/table/tbody/tr[6]/td//text()')
        Music = element_tree.xpath('//*[@id="mw-content-text"]/div[1]/table/tbody/tr[7]/td//text()')
        Relasedate = element_tree.xpath('//*[@id="mw-content-text"]/div[1]/table[1]/tbody/tr[9]/td/div/ul/li/text()')
        Runtime = element_tree.xpath('//*[@id="mw-content-text"]/div[1]/table/tbody/tr[12]/td//text()')
        country = element_tree.xpath('//*[@id="mw-content-text"]/div[1]/table/tbody/tr[13]/td/text()')
        language = element_tree.xpath('//*[@id="mw-content-text"]/div[1]/table/tbody/tr[14]/td//text()')
        Budget = element_tree.xpath('//*[@id="mw-content-text"]/div[1]/table/tbody/tr[17]/td/text()')
        print(p)
        print(Directed,Produced,Written,Starring,Music,Relasedate,Runtime,country,language,Budget, end = '\n')
        print('*'*100)
    except Exception as ex:
        print('error is ', ex)


https://en.wikipedia.org/wiki/Z.P.G.
['Michael Campus'] ['Frank De Felitta', 'Max Simon Ehrlich', 'Tom Madigan'] ['Frank De Felitta', 'Max Simon Ehrlich'] ['Oliver Reed', 'Geraldine Chaplin', 'Don Gordon', 'Diane Cilento'] ['Jonathan Hodge'] [] ['97 minutes'] ['Denmark', 'United States'] ['English'] []
****************************************************************************************************
https://en.wikipedia.org/wiki/Zee_and_Co.
['Brian G. Hutton'] ['Elliot Kastner', 'Jay Kanter', 'Alan Ladd, Jr.'] ["Edna O'Brien"] ['Elizabeth Taylor', 'Michael Caine', 'Susannah York'] ['Stanley Myers'] [] ['21 January 1972'] ['110 minutes'] ['United Kingdom'] []
****************************************************************************************************
https://en.wikipedia.org/wiki/And_Now_the_Screaming_Starts!
['Roy Ward Baker'] ['Max J. Rosenberg', 'Milton Subotsky'] ['Roger Marshall'] ['Fengriffen', ' (novella)', 'by ', 'David Case'] ['Peter Cushing', 'Herbert Lom', 'Patrick 

# Table information with bs4

In [135]:
response = requests.get('https://en.wikipedia.org/wiki/Z.P.G.') 
soup = BeautifulSoup(response.text, 'html.parser')
table = soup.find('table', class_ = 'infobox vevent')

In [145]:
for x in table.find_all('tr')[2:]:
    for i in x.find_all('td'):
        print(i.text,end='\n')

Michael Campus
Frank De FelittaMax Simon EhrlichTom Madigan
Frank De FelittaMax Simon Ehrlich
Oliver ReedGeraldine ChaplinDon GordonDiane Cilento
Jonathan Hodge
Michael ReedMikael Salomon
Dennis Lanning
Scotia-Barber

25 May 1972 (1972-05-25)

97 minutes
DenmarkUnited States[1]
English


In [220]:
check_list = ['Directed by','Produced by','Written by','Starring','Music by','Release date','Running time','Countries',
              'Language','Budget']
d={}
for x in table.find_all('tr')[2:]:
    if x.th.text in check_list:
        for i in x.find_all('th'):
            for y in x.find_all('td'):
                d[i.text]=y.text
                
for i in check_list:
    if i not in d.keys():
        d[i]= 'NA'
print(d)

{'Directed by': 'Michael Campus', 'Produced by': 'Frank De FelittaMax Simon EhrlichTom Madigan', 'Written by': 'Frank De FelittaMax Simon Ehrlich', 'Starring': 'Oliver ReedGeraldine ChaplinDon GordonDiane Cilento', 'Music by': 'Jonathan Hodge', 'Release date': '\n25\xa0May\xa01972\xa0(1972-05-25)\n', 'Running time': '97 minutes', 'Countries': 'DenmarkUnited States[1]', 'Language': 'English', 'Budget': 'NA'}


In [ ]:
# Release date': '\n25\xa0May\xa01972\xa0(1972-05-25)\n'
# how to fix ???